In [3]:
# import libraries
import requests
import json
from dataclasses import dataclass, asdict
from typing import List
import pandas as pd

In [4]:
# Define models
@dataclass
class Order:
    symbol: str
    qty: int
    side: str
    type: str
    price: float
    time_in_force: str
        
@dataclass
class Trade:
    symbol: str
    qty: int
    price: float
    side: str
    timestamp: str
    order_id: str
    instmt: str
    trade_price: str
    trade_qty: str
    trade_side: str
    trade_id: int
    @classmethod
    def from_dict(cls, data: dict) -> 'Trade':
        return cls(**data)
    
@dataclass
class StockDetails:
    symbol: str
    name: str
    exchange: str
    currency: str
    country: str
    sector: str
    industry: str
    market_cap: float
    beta: float
    pe_ratio: float
    eps: float
    dividend_yield: float
    dividend_per_share: float
    dividend_pay_date: str
    dividend_ex_date: str
    last_split_factor: str
    last_split_date: str    
    @classmethod
    def from_dict(cls, data: dict) -> 'StockDetails':
        return cls(**data)
      
@dataclass
class TopStock:
    symbol: str
    name: str
    change_pct: float
    price: float

In [5]:
# Define the base URL of the dummy stock exchange API
base_url = 'http://localhost:5000/api/'

# services
def get_stock_detail(symbol)-> StockDetails:
    endpoint = 'v1/stock/'+symbol
    response = requests.get(base_url+ endpoint)
    if response.status_code == 200:
        stock_dict = response.json()
        stock = StockDetails.from_dict(stock_dict)
        return stock
    else:
        raise Exception("Internal server error")
    
def place_order(order) -> Trade:
    endpoint = 'v1/orders/place'
    response = requests.post(base_url + endpoint, json=asdict(order))

    if response.status_code == 200:
        print("Order Placed Successfully")
        respdict = response.json()
        trade = Trade.from_dict(respdict)
        return trade
    else:
        raise Exception("Order was not executed")
        
def cancel_order(order):
    # Define the endpoint for placing an order
    endpoint = 'v1/orders/cancel'
    # Send the request to place the order
    response = requests.delete(base_url + endpoint, json=asdict(order))
    # Print the response
    print(response.status_code)
    
    # Print the trade response
    if response.status_code == 200:
        trade_dict = response.json()['trade']
        print("Order deleted")
    else:
        raise Exception("Order was not cancelled")

In [6]:
tradersTop10Stocks = ['AAPL','MSFT','GOOG','AMZN','PCAR','TSLA','META']

In [7]:
# Program should first read the list of instruments (with relevant details) available on the exchange. 
stocks = [get_stock_detail(tradersTop10Stocks[0])]

In [8]:
from prettytable import PrettyTable

# Assuming you have a list of StockDetails objects called 'stocks'
table = PrettyTable()
table.field_names = ["Symbol", "Name", "Exchange", "Currency", "Country", "Sector", "Industry", "Market Cap", "Beta", "P/E Ratio", "EPS", "Dividend Yield", "Dividend/Share", "Dividend Pay Date", "Dividend Ex Date", "Last Split Factor", "Last Split Date"]

for stock in stocks:
    table.add_row([stock.symbol, stock.name, stock.exchange, stock.currency, stock.country, stock.sector, stock.industry, stock.market_cap, stock.beta, stock.pe_ratio, stock.eps, stock.dividend_yield, stock.dividend_per_share, stock.dividend_pay_date, stock.dividend_ex_date, stock.last_split_factor, stock.last_split_date])

print(table)


+--------+------------+----------+----------+---------+------------+----------------------+------------+------+-----------+------+----------------+----------------+-------------------+------------------+-------------------+-----------------+
| Symbol |    Name    | Exchange | Currency | Country |   Sector   |       Industry       | Market Cap | Beta | P/E Ratio | EPS  | Dividend Yield | Dividend/Share | Dividend Pay Date | Dividend Ex Date | Last Split Factor | Last Split Date |
+--------+------------+----------+----------+---------+------------+----------------------+------------+------+-----------+------+----------------+----------------+-------------------+------------------+-------------------+-----------------+
|  AAPL  | Apple Inc. |  NASDAQ  |   USD    |   USA   | Technology | Consumer Electronics | 2234926.77 | 1.22 |   30.92   | 5.19 |      0.61      |      0.88      |     2023-03-11    |    2023-03-14    |        7:1        |    2023-03-11   |
+--------+------------+---------

In [9]:
order = Order(symbol='AAPL', qty=100, side='buy', type='limit', price=150.0, time_in_force='gtc')
trade = place_order(order)

Order Placed Successfully


In [10]:
# Assuming you have a list of StockDetails objects called 'stocks'
table = PrettyTable()
table.field_names = ["Symbol", "Quantity", "Price", "Side", "OrderPlacedAt",
                     "OrderId", "Stock","Trade Price","Trade Quantity","Trade Side","TradeId"]
table.add_row([trade.symbol, trade.qty, trade.price, trade.side, trade.timestamp, trade.order_id, trade.instmt,
               trade.trade_price,  trade.trade_qty,trade.trade_side,trade.trade_id])
print(table)

+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+
| Symbol | Quantity | Price | Side |       OrderPlacedAt        |               OrderId                | Stock | Trade Price | Trade Quantity | Trade Side |               TradeId                |
+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+
|  AAPL  |   100    | 150.0 | buy  | 2023-03-13T04:45:02.157227 | fb2d920f-69d2-48b5-88d8-742682d6ed1a |  AAPL |    150.0    |      100       |    buy     | a6358ca3-38a5-419f-8559-e5d188ad901b |
+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+


<h3>Generate orders in bulk</h3>

In [12]:
import random
import datetime

# Define a list of stocks and their historical prices
stock_prices = {
    'AAPL': [150, 152, 151, 149, 153, 155, 156, 157, 159, 160],
    'GOOG': [1200, 1210, 1220, 1205, 1195, 1185, 1190, 1180, 1175, 1165],
    'TSLA': [700, 710, 720, 730, 735, 740, 745, 750, 755, 760],
    'AMZN': [3000, 3010, 3020, 3015, 3012, 3014, 3018, 3016, 3013, 3011],
    'FB': [350, 355, 360, 365, 370, 375, 380, 385, 390, 395]
}

# Define a list of possible sides
sides = ['BUY', 'SELL']

# Define a list of possible quantities
quantities = list(range(1, 11))

# Define a function to generate a random order
def generate_order():
    # Generate a random stock symbol from the list
    stock = random.choice(list(stock_prices.keys()))

    # Get the last price of the stock
    last_price = stock_prices[stock][-1]

    # Generate a random quantity between 1 and 10
    quantity = random.choice(quantities)

    # Generate a random buy or sell side
    side = random.choice(sides)

    # Generate a realistic price based on the last price of the stock
    if side == 'BUY':
        # Add a random percentage to the last price
        price = round(last_price * (1 + random.uniform(0, 0.05)), 2)
    else:
        # Subtract a random percentage from the last price
        price = round(last_price * (1 - random.uniform(0, 0.05)), 2)

    # Get the current time
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Create the order dictionary using the randomly generated values
    order = Order(symbol=f'{stock}', qty=quantity, side=f'{side}', type='limit', price=price, time_in_force=timestamp)  
    return order

# Generate 100 random orders
for i in range(100):
    # Generate a random order
    order = generate_order()
    # Print the order statement
    print(f"Randomly generated order statement {i+1}: {order}")    
    trade = place_order(order)
    print(trade)

Randomly generated order statement 1: Order(symbol='AAPL', qty=8, side='SELL', type='limit', price=154.51, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='AAPL', qty=8, price=154.51, side='SELL', timestamp='2023-03-13T04:50:12.128211', order_id='eaeaf653-8fc9-4737-8cf0-e2f8a856feb1', instmt='AAPL', trade_price=154.51, trade_qty=8, trade_side='SELL', trade_id='e65a057e-22cd-4a49-b23f-1e176d9171b5')
Randomly generated order statement 2: Order(symbol='AAPL', qty=1, side='BUY', type='limit', price=162.41, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='AAPL', qty=1, price=162.41, side='BUY', timestamp='2023-03-13T04:50:12.134204', order_id='a0df3137-f3db-47bc-9712-c308c81b1881', instmt='AAPL', trade_price=162.41, trade_qty=1, trade_side='BUY', trade_id='65506e8e-71d2-4cdb-9424-d356f4146f0f')
Randomly generated order statement 3: Order(symbol='TSLA', qty=6, side='SELL', type='limit', price=752.62, time_in_force='2023-03-13 00:50:12')

Order Placed Successfully
Trade(symbol='AAPL', qty=4, price=165.22, side='BUY', timestamp='2023-03-13T04:50:12.329305', order_id='4d59a3d6-a1b8-4cd5-a531-49de69492b25', instmt='AAPL', trade_price=165.22, trade_qty=4, trade_side='BUY', trade_id='99363707-b3ae-402e-8dde-9bb45ff79b40')
Randomly generated order statement 40: Order(symbol='FB', qty=5, side='SELL', type='limit', price=386.73, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='FB', qty=5, price=386.73, side='SELL', timestamp='2023-03-13T04:50:12.334031', order_id='26966d11-a6ca-4f1c-a355-ce0a9da817ab', instmt='FB', trade_price=386.73, trade_qty=5, trade_side='SELL', trade_id='617a03b8-11e1-41dc-ac97-d7fc998b4932')
Randomly generated order statement 41: Order(symbol='FB', qty=3, side='BUY', type='limit', price=396.68, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='FB', qty=3, price=396.68, side='BUY', timestamp='2023-03-13T04:50:12.340855', order_id='95186860-15b0-4eca-a4

Order Placed Successfully
Trade(symbol='AAPL', qty=7, price=156.45, side='SELL', timestamp='2023-03-13T04:50:12.535691', order_id='76860d99-370b-4621-a0c6-0254982ea78f', instmt='AAPL', trade_price=156.45, trade_qty=7, trade_side='SELL', trade_id='7227cc68-c8d1-4462-bef6-cc1e0ee92eb3')
Randomly generated order statement 73: Order(symbol='TSLA', qty=4, side='BUY', type='limit', price=771.15, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='TSLA', qty=4, price=771.15, side='BUY', timestamp='2023-03-13T04:50:12.541750', order_id='60398162-86ed-401f-a382-b2e3cf33645d', instmt='TSLA', trade_price=771.15, trade_qty=4, trade_side='BUY', trade_id='4528506f-363a-4cd0-8e6c-ba21f87694b3')
Randomly generated order statement 74: Order(symbol='FB', qty=5, side='BUY', type='limit', price=399.98, time_in_force='2023-03-13 00:50:12')
Order Placed Successfully
Trade(symbol='FB', qty=5, price=399.98, side='BUY', timestamp='2023-03-13T04:50:12.547033', order_id='78f88f84-3fc3-4b